In [ ]:
# ! pip install openai
# ! pip install openai openpyxl
# ! pip install langchain
# ! pip install tiktoken

In [1]:
import os
import pandas as pd
from langchain.llms import OpenAI
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)
from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain

In [2]:
api_key = os.environ.get('OPENAI_API_KEY')

In [4]:
# Initialize the LLM
llm = OpenAI(model_name='davinci-002', api_key=api_key)

In [5]:
# Read the CSV file containing questions and answers
csv_file = 'C:/Users/Owner/Desktop/CGU/IST 498 - Generative_AI_Independent_Study/Large Language Model/CGU_QA.csv'
data = pd.read_csv(csv_file)

In [6]:
data.head()

,Question,Answer,Topic
0,Do you offer information sessions?,Yes. We regularly offer sessions and prospecti...,Admissions Process
1,Is there an advantage to applying by the prior...,Applicants who apply by the priority deadline ...,Admissions Process
2,What are the application requirements?,Application requirements(https://www.cgu.edu/a...,Admissions Process
3,When will I find out my admission decision?,After you submit a completed application for a...,Admissions Process
4,Is a master’s degree a prerequisite for admiss...,This varies by area of study. Many programs gi...,Admissions Process


In [7]:
# Create a knowledge base (dictionary) that combines answers based on the topic
knowledge_base = {}
for _, row in data.iterrows():
    topic = row['Topic']
    question = row['Question']
    answer = row['Answer']
    
    if topic not in knowledge_base:
        knowledge_base[topic] = []
    knowledge_base[topic].append(f"Q: {question}\nA: {answer}\n")

In [8]:
# Combine answers for each topic
for topic in knowledge_base:
    knowledge_base[topic] = "\n".join(knowledge_base[topic])

In [9]:
# Function to get the combined answer based on the topic
def get_combined_answer(topic):
    return knowledge_base.get(topic, "I'm sorry, I don't have information on that topic.")

In [10]:
# Initialize the ChatOpenAI model
chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.3)

In [11]:
# Define the system message for the chatbot
system_message = SystemMessage(content="You are an expert educator, providing clear and accurate answers to students' questions.")

In [26]:
# Create the first prompt template for generating detailed answers
first_template = """
You are an expert educator with extensive knowledge in various subjects.
Answer the following questions comprehensively based on the provided information.
Topic: {input}
"""
first_prompt = PromptTemplate(
    input_variables=["input"],
    template=first_template,
)

In [27]:
# Create the second prompt template for simplifying the detailed answer
second_template = """
Now, simplify the explanation of the topic:
{input}
"""
second_prompt = PromptTemplate(
    input_variables=["input"],
    template=second_template,
)

In [28]:
# Create LLMChains for both templates
first_chain = LLMChain(llm=llm, prompt=first_prompt, output_key="detailed_answer")
second_chain = LLMChain(llm=llm, prompt=second_prompt)

In [29]:
# Combine the chains into a SimpleSequentialChain
overall_chain = SimpleSequentialChain(chains=[first_chain, second_chain], verbose=True)

In [30]:
# Function to get a response from the chatbot based on topic
def get_response(topic):
    # Get the combined answers for the topic
    combined_answers = get_combined_answer(topic)
    
    # If the topic is not found in the knowledge base, return the default response
    if combined_answers == "I'm sorry, I don't have information on that topic.":
        return combined_answers
    
    # Concatenate topic and combined answers into a single input
    input_text = f"Topic: {topic}\n{combined_answers}"
    
    # Run the overall chain specifying only the input variable for the first chain
    response = overall_chain.run(input_text)
    return response

In [31]:
# Example usage
topic = "Deep Learning"
response = get_response(topic)
print(response)

I'm sorry, I don't have information on that topic.


In [32]:
# Example usage
topic = "Admissions Process"
response = get_response(topic)
print(response)



> Entering new SimpleSequentialChain chain...
Q: What if I don’t meet the minimum requirements?
A: We want to help you succeed! Contact your admissions representative(https://www.cgu.edu/admissions/meet-our-team/) to discuss your admission options.

Q: Will my application be considered if my application is incomplete?
A: We encourage you to submit a complete application as soon as possible. If an application is incomplete, the application will remain pending until all required materials are received. As a reminder, all application materials must be received by the posted deadline.

Q: How can I find out more about the Admissions Committee’s decision?
A: The Admissions Committee does not share its deliberations or provide comments on individual applications. The Admissions Committee makes its decisions based on the application as a whole. We recommend you use the opportunity to reapply.

Q: How can I check the status of my application?
A: You can check the status of your application b